# Assignment XML processing

## Notebook made by  

|** Name** | **Student id** | **email**|
|:- |:-|:-|
|. | | |
|  | |. |

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. 

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='link to your selfie'/>

### Note
* **Assignments without the selfies or completely filled in information will not be graded and receive 0 points.**


# XML opdracht

* We gaan bekijken hoe vaak Wilders onderbroken wordt in de Tweede Kamer en door wie.
* Als input files hebben we de Handelingen van de Tweede Kamer in keurig XML formaat.
* Je kunt die bekijken op [overheid.nl](https://zoek.officielebekendmakingen.nl/zoeken/resultaat?zkt=Uitgebreid&pst=ParlementaireDocumenten&vrt=opening&zkd=AlleenInDeTitel&dpr=AnderePeriode&spd=20140101&epd=20160930&kmr=TweedeKamerderStatenGeneraal&sdt=KenmerkendeDatum&par=Handeling&dst=Onopgemaakt%7cOpgemaakt%7cOpgemaakt+na+onopgemaakt&isp=true&pnr=1&rpp=10)

* Jouw taak is code te schrijven die 2 dingen doet:


### Data voor de opdracht

<http://maartenmarx.nl/teaching/DataScience/Data/WildersXMLassignment>

## Taak 1
Maak uit de XML bestanden in `TK_Handelingen_2013-2016.zip`  een bestand `interrupties.csv` met de volgende soort data: de kolommen zijn
`'date','termijn','spreker','rol','indegree','aantal_interrupts','interruptors','interruptie_volgorde','OnderwerpDebat','url'` en dit is een voorbeeld.
   
    
```
20122013-10-15,1,Recourt,De heer,3,8,Van Raak-Van Raak-Van Raak-Pechtold-Pechtold-Pechtold-Van Ojik-Van Ojik,Begroting Algemene Zaken en begroting van de Koning,https://zoek.officielebekendmakingen.nl/h-tk-20122013-10-15.html 
20122013-10-15,1,Van der Burg,Mevrouw,2,6,Thieme-Thieme-Pechtold-Pechtold-Pechtold-Pechtold,Begroting Algemene Zaken en begroting van de Koning,https://zoek.officielebekendmakingen.nl/h-tk-20122013-10-15.html 
20122013-10-15,1,Bosma,De heer,0,0,,Begroting Algemene Zaken en begroting van de Koning,https://zoek.officielebekendmakingen.nl/h-tk-20122013-10-15.html 
20122013-10-15,1,Van Toorenburg,Mevrouw,2,5,Pechtold-Pechtold-Pechtold-Bosma-Bosma,Begroting Algemene Zaken en begroting van de Koning,https://zoek.officielebekendmakingen.nl/h-tk-20122013-10-15.html 
20122013-10-15,1,Van Raak,De heer,2,6,Recourt-Van der Burg-Van der Burg-Van der Burg-Recourt-Recourt,Begroting Algemene Zaken en begroting van de Koning,https://zoek.officielebekendmakingen.nl/h-tk-20122013-10-15.html 
```
* `termijn` is een integer dat de keer in het debat aangeeft dat de spreker vanaf het spreekgestoelte de zaal toespreekt. In de Kamer spreekt men van "eerste termijn" en "tweede termijn".
* `indegree` betekent hier het aantal mensen dat `spreker` onderbroken heeft. `aantal_interrupts` is het totaal aantal keer dat een spreker onderbroken is.
* Let op, de `date` kolom bevat niet echt een datum, maar is gemaakt van de filenaam: het is dus de periode, gevolgd door de hoeveelste vergadering in die periode, gevolgd het hoeveelste debat in die vergadering. 
* Let op, de filenamen zijn net ietsje anders dan deze waardes. Op deze manier is een eenvoudige sort voldoende om ze netjes chronologisch te krijgen.

#### Let op
Je zal zelf moeten uitvogelen hoe interrupties en sprekers etc in de XML zijn gecodeerd. Doe dit door de PDFs en HTML te bekijken, en desnoods een video van een debat. 
    
## Taak 2

Maak uit `interrupties.csv` met Pandas een dataframe wat er net zo uitziet als `Interruptors_van_Wilders.csv `.
Die staat ook op <https://docs.google.com/spreadsheets/d/1PCOMeKV4-Y6_hZnN0o7-HxmrwSVLcP-ZP9vQK0DFLP8/edit#gid=49831024>


## Taak 3

* Plot het aantal interrupties van Wilders per debat samen met het gemiddeld aantal interrupties in dat debat.

## Taak 4
* Verzin zelf iets geks met deze data en laat dat in een leuk plotje zien.


# Let op

* Maak je code zo dat **Wilders** een variabele wordt die aan het begin wordt gezet.
* In jouw code moet ik die variabele kunnen vervangen door **Pechtold** of **Samson** of wie dan ook, en een vergelijkbaar spreadsheet uit taak 3 (en hopelijk ook 4) krijgen. 

In [38]:
# Imports
import pandas as pd
import numpy as np
import glob
import os
import xml.etree.ElementTree as ET

print 'Slight change!'
# Surpress Pandas warning.
pd.options.mode.chained_assignment = None

# Initialize Pandas dataframe and series
col     = ['date','termijn','spreker','rol','indegree','aantal_interrupts','interruptors','interuptie_volgorde','OnderwerpDebat','url']
df      = pd.DataFrame(columns=col)
current = pd.Series(index=col)

# Function to process each file
def parse_file(path): 
    # get xml tree
    tree = ET.parse(path)
    root = tree.getroot()
    
    # set url
    for elem in tree.iter(tag='meta'):
        url = elem.get('content')
        current.url = url[:-13] + '.html'
    # set date based on filename
    current.date = str(path[17:-4])
    
    # loop through each agendapunt.
    for elem in tree.iter(tag='agendapunt'):
        # set onderwerp debat
        current.OnderwerpDebat = get_tag(elem,'item-titel')[0]
        try: get_sprekers(elem)
        except: continue


# help function to retrieve info
def get_tag(tree,tag): 
    li = []
    for elem in tree.iter(tag=tag):
        li.append(elem.text)
    return li

    

# add gathered information to existing dataframe
def process(sprekers):
    global df
    if len(sprekers) > 0 and 'voorzitter' not in sprekers[0]:
        current.spreker = sprekers[0]
        interruptors = [x for x in sprekers if 'voorzitter' not in x and x != current.spreker]
        current.termijn           = len(df[(df.spreker == current.spreker) & (df.date == current.date) & (df.OnderwerpDebat == current.OnderwerpDebat)]) + 1
        current.interruptors = dict( [ (i, interruptors.count(i)) for i in set(interruptors) ] )
        current.indegree          = len(set(interruptors))
        current.aantal_interrupts = len(interruptors)
        current.interruptors = interruptors_to_string(interruptors)
        df = df.append(current, ignore_index=True)
  
# Function to convert list of interruptors to the right string format
def interruptors_to_string(interruptors): 
    if len(interruptors) > 0: 
        string = interruptors[0]
        for elem in interruptors[1:]:
            string += "-" + elem
        return string
    else: 
        return ''
        
# Function to get all sprekers in a given subtree.  
def get_sprekers(subtree): 
    sprekers = []
    # loop through each 'spreekbeurt'
    for elem in subtree.iter(tag='spreekbeurt'): 
        # if new: add previously gathered information to dataframe
        if elem.get('nieuw') == 'ja': 
            process(sprekers) 
            sprekers = []
            current.rol = get_tag(elem,'voorvoegsels')[0]
            sprekers.append(get_tag(elem,'achternaam')[0])
        else: 
            sprekers.append(get_tag(elem,'achternaam')[0])
    process(sprekers) # last to ensure last info gets added
    
print 'Cell ran succesfully!'

In [39]:
# Taak 1:
for path in glob.glob('Wilders2014/*.xml'):
    parse_file(path)
df.to_csv('test.csv',encoding='utf-8')

    
print 'Cell ran succesfully!'
    

    date  termijn           spreker      rol  indegree  aantal_interrupts  \
0  3-3-8      1.0        Mark Rutte  De heer       7.0               36.0   
1  3-3-8      1.0            Samsom  De heer       6.0               28.0   
2  3-3-8      1.0           Wilders  De heer       0.0                0.0   
3  3-3-8      1.0            Roemer  De heer       1.0                4.0   
4  3-3-8      1.0  Van Haersma Buma  De heer       0.0                0.0   

                                        interruptors interuptie_volgorde  \
0  Pechtold-Pechtold-Pechtold-Roemer-Roemer-Roeme...                 NaN   
1  Roemer-Roemer-Roemer-Van Haersma Buma-Van Haer...                 NaN   
2                                                                    NaN   
3                        Samsom-Samsom-Samsom-Samsom                 NaN   
4                                                                    NaN   

             OnderwerpDebat                                                url  

In [45]:
# Taak 2: 
def spreker_interrupties(interrupted_spreker):
    df = pd.read_csv('interrupties.csv')
    sub_df = df[df.spreker == interrupted_spreker]
    li = [] 

    # create list of *all* interrupting people -> keys
    for i,row in sub_df.iterrows(): 
        interruptors = row.interruptie_volgorde
        if type(interruptors) == str: 
            li = li + interruptors.split('-')
    keys = sorted(list(set(li)))

    for i, row in sub_df.iterrows():
        # NOTE: numbers do not seem to match with example file. Wrong definition of 'debat'? Assumed same date + same subject
        # retrieve means
        debat_df = df[(df.date == row.date) & (df.OnderwerpDebat == row.OnderwerpDebat)]
        aantal_interrupts_mean = np.mean(debat_df.aantal_interrupts.tolist())
        indegree_mean = np.mean(debat_df.indegree.tolist())

        # add to consisting sub dataframe containing only interrupted_spreker as spreker
        sub_df.loc[row.name,'aantal_interrupts_mean'] = aantal_interrupts_mean
        sub_df.loc[row.name,'indegree_mean'] = indegree_mean

        # retrieve interrupting individuals counts
        dictionary = dict(zip(keys,[0]*len(keys)))
        interrupties = row.interruptie_volgorde
        if type(interrupties) == str:
            for elem in interrupties.split('-'): 
                dictionary[elem] = dictionary[elem] + 1
            for k in keys: 
                sub_df.loc[row.name,k] = dictionary[k]
        else: 
            for k in keys:
                sub_df.loc[row.name,k] = ''
                
    sub_df.to_csv('interrupties_van_%s.csv'%interrupted_spreker,encoding='utf-8')
    
spreker_interrupties('Wilders')
spreker_interrupties('Samsom')
print 'Cell ran succesfully!'

     Unnamed: 0            date  termijn spreker      rol  indegree  \
39           39  20122013-100-3      1.0  Samsom  De heer       7.0   
52           52  20122013-100-6      1.0  Samsom  De heer       4.0   
918         918   20122013-17-6      1.0  Samsom  De heer      21.0   
942         942   20122013-17-9      1.0  Samsom  De heer       0.0   
963         963   20122013-21-4      1.0  Samsom  De heer       5.0   

     aantal_interrupts                                       interruptors  \
39                53.0  {u'Roemer': 13, u'Van Ojik': 6, u'Van Haersma ...   
52                12.0  {u'Rutte': 1, u'Thieme': 3, u'Van Haersma Buma...   
918              107.0  {u'Van Haersma Buma': 15, u'Roemer': 24, u'Van...   
942                0.0                                                 {}   
963               60.0  {u'Van Haersma Buma': 18, u'Roemer': 11, u'Slo...   

                                  interruptie_volgorde  \
39   Pechtold-Pechtold-Pechtold-Van Haersma Buma-Va.

In [7]:
# some problems with running matplotlib in a virtual env.. 
import matplotlib.pyplot as plt

RuntimeError: Python is not installed as a framework. The Mac OS X backend will not be able to function correctly if Python is not installed as a framework. See the Python documentation for more information on installing Python as a framework on Mac OS X. Please either reinstall Python as a framework, or try one of the other backends. If you are Working with Matplotlib in a virtual enviroment see 'Working with Matplotlib in Virtual environments' in the Matplotlib FAQ

In [8]:
# Taak 3: 
df = pd.read_csv('interrupties_van_wilders.csv')
interrupties_spreker = df.aantal_interrupts.tolist()
interrupties_mean = df.aantal_interrupts_mean.tolist()
onderwerpen = df.OnderwerpDebat # retrieve onderwerpen to use as x-axis labels - long names makes messy plot, should turn names sideways.

# Taak 3: Simple scatter plot with interruptions for Wilders vs. mean interruptions per debate.
x = list(xrange(len(interrupties_spreker)))
y = interrupties_spreker
z = interrupties_mean

plt.scatter(x,y, color='red')
plt.scatter(x,z, color='blue')

plt.show()

print 'Cell ran succesfully!'


NameError: name 'plt' is not defined

In [ ]:
# Taak 3: Simple bar chart with interruptions for Wilders vs. mean interruptions per debate.
N = len(interrupties_spreker)
x = tuple(interrupties_spreker)
y = tuple(interrupties_mean)

ind = np.arange(N)  # the x locations for the groups
width = 0.35      # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(ind, x, width, color='r')
rects2 = ax.bar(ind + width, y, width, color='y')
ax.set_ylabel('Interrupties')
ax.set_title('Interrupties per debate: %s and mean.'%interrupted_spreker)
ax.set_xticks(ind + width)

ax.set_xticklabels(tuple())
ax.legend((rects1[0], rects2[0]), (interrupted_spreker, 'Mean'))

plt.show()

print 'Cell ran succesfully!'

In [ ]:
# Taak 4: show bar chart of number of times the speaker has been interrupted and by whom.

def create_bar(spreker): 
    df = pd.read_csv('interrupties_van_%s.csv'%spreker)
    col_index = df.columns.get_loc('indegree_mean') # look at all columns after indegree_mean
    names = []
    values = []
    p = [0] * 2
    ind = np.arange(len(df.columns)-col_index+1)
    width = 0.35
    
    # for each column: get the column name and sum of interruptions. 
    for i in xrange(col_index+1, len(df.columns)): 
        series = df.ix[:,i]
        names.append(series.name)
        values.append(np.nansum(series.tolist()))

    # quick unicode problem fix - names with unusual characters can't be printed in the graph, so temporarily throw them out.. 
    for i in xrange(len(names)): 
        try: 
            print str(names[i])
        except: 
            del names[i]
            del values[i]

    width = 1/1.5
    p[0] = plt.bar(range(len(values)), tuple(values), width, color="blue")
    plt.ylabel('Nr. of interruptions')
    plt.xticks(ind+width/2., tuple(names))
    plt.yticks(np.arange(0, 81, 10))
    plt.title('%s interrupted by: '%spreker)
    plt.show()
    
create_bar('Wilders')

print 'Cell ran succesfully!'

In [ ]:
''' 
Taak 4: Some additional things. Probably doesn't work yet, but I can't test whether it does. 
The intention is to create a stacked bar chart for each individual, where the X-axis represents the interruptors
and the colors represent who they have interrupted (could be the other way around, but there are more interruptors
than speakers, so that could produce a messy, unclear chart). The size of the bar represents how often the interruptor
has interrupted the interruptee - this could provide some insight into who the 'biggest offenders' are with regards to
interrupting speakers, but should also give some insight as to whether these offenders are disproportionally targetting
specific speakers or generally more inclined to interrupt than others.

Note: currently will not produce correct result as spreker_interrupties() only concatinates columns to the sub dataframe
for people who have, actually, interrupted said speaker. Therefore, the columns in the various files do not match, even
though that is the current assumption. Simply combining and stacking the various 'values' from the columns will therefore
produce skewed results. However, this should be easy to solve by either concatinating the various sub_df's from the read
in files, or creating keeping track of the totals for all interruptors.
'''

# returns the amount of time each interruptor has interrupted the speaker
def return_namesvalues(ds): 
    col_index = ds.columns.get_loc('indegree_mean')
    names = []
    values = []
    p = [0] * 2
    ind = np.arange(len(df.columns)-col_index+1)
    width = 0.35
    for i in xrange(col_index+1, len(df.columns)): 
        series = df.ix[:,i]
        names.append(series.name)
        values.append(np.nansum(series.tolist()))

    for i in xrange(len(names)): 
        try: 
            print str(names[i])
        except: 
            del names[i]
            del values[i]
    return names,values

# -- script -- # 
unique_sprekers = list(set(df.spreker.tolist()))
y_offset = []
p = [0] * len(unique_sprekers)
width=1/1.5
for i in xrange(len(unique_sprekers)): 
    if i < 2:
        elem = unique_sprekers[i]
        print elem
        if not os.path.exists('interrupties_van_%s.csv'%elem): 
            spreker_interrupties(elem)
        
        names, values = return_namesvalues(pd.read_csv('interrupties_van_%s.csv'%elem))
        if i == 0: 
            p[0] = plt.bar(range(len(values)),values,width)
            y_offset = values
        else: 
            p[0] = plt.bar(range(len(values)),values,width,bottom=y_offset)
            y_offset = values + y_offset
            
plt.legend(p, unique_sprekers)
plt.ylabel('Interrupties')
plt.xticks(ind+width/2., tuple(names))
plt.yticks(np.arange(0, 81, 10))
plt.show()

print 'Cell ran succesfully!'